# Example: Writing a thermal network model to the extended 3DCityDB

This example shows how to write a simple thermal network model to the extended *3DCityDB*, using the `dblayer` package. Basic knowledge about [CityGML](http://www.citygmlwiki.org) and the [Utility Network ADE](https://en.wiki.utilitynetworks.sig3d.org) are required for this notebook. A nice overview of the of the basic concepts of the Utility Network ADE can be found [here](https://en.wiki.utilitynetworks.sig3d.org/images/upload/20160913_-_Kolbe_-_Modeling_3D_Utility_Networks_%2B_UtilityNetwork_ADE_Core_Model.pdf).

For the example, a simple thermal network is used, as shown in the following figure (SRC = thermal source, SNK = thermal sink, N = network node / pipe junction).

<img src="./figures/thermal_network.svg" style="width: 650px;">

## Getting started ...

Import the required funtionality from package `dblayer`.

In [1]:
from dblayer.helpers.utn.thermal_network import *

Load data from CSV files, containing information about busses, lines and loads.

In [2]:
import pandas as pd
import os.path

sinks_csv_data = pd.read_csv( os.path.join( 'csv', 'thermal_sink_data.csv' ), index_col = 0 )
junctions_csv_data = pd.read_csv( os.path.join( 'csv', 'thermal_junction_data.csv' ), index_col = 0 )
pipes_csv_data = pd.read_csv( os.path.join( 'csv', 'thermal_pipe_data.csv' ), index_col = 0 )

Dict for most relevant data of thermal nodes (i.e., thermal sources, thermal loads, pipe junctions) added to the database 

In [3]:
nodes_data = {}

## Connect to database

Define connection parameters for 3DCityDB instance.

In [4]:
connect = PostgreSQLConnectionInfo(
    user = 'postgres',
    pwd = 'postgres',
    host = 'localhost',
    port = '5432',
    dbname = 'citydb'
    )

Connect to database using an instance of class `DBAccess`.

In [5]:
db_access = DBAccess()
db_access.connect_to_citydb( connect )

For the purpose of this notebook yoi might want to use an empty 3DCityDB. To this end, the next two lines erase all data from an existing database. **Only execute the next two lines if you really want to erase all data from your database!**

In [6]:
db_access.cleanup_citydb_schema()
db_access.cleanup_simpkg_schema()

## Write the network to the database
Define the spatial reference ID (SRID) of the network. This should be consistent with the setup of your database.

In [7]:
srid = 4326

Insert a new *network* and an associated *network graph*. The *network* basically contains the topographical representation - the so-called *network features* - of all the network network components. The *network graph* contains the toplogical representation - of all these components the topological links between them. Use function `write_network_to_db` from package `dblayer.helpers.utn.thermal_network`.

In [8]:
( ntw_id, ntw_graph_id ) = write_network_to_db(
    db_access,
    name = 'thermal_example_network',
    id = 2000
    )

Add thermal source to the network. Use function `write_terminal_element_to_db` from package `dblayer.helpers.utn.thermal_network`.

In [9]:
source_name = 'source-SRC'

nodes_data[source_name] = write_terminal_element_to_db( 
    db_access,
    source_name,
    Point2D( 0., 0. ),
    'thermal-source',
    srid,
    ntw_id,
    ntw_graph_id
    )

Add thermal sinks to the network. Use function `write_dhw_sink_to_db` from package `dblayer.helpers.utn.thermal_network`.

In [10]:
for sink_name, sink_csv_data in sinks_csv_data.iterrows():

    nodes_data[sink_name] = write_dhw_sink_to_db(
        db_access,
        sink_name,
        Point2D( sink_csv_data.x, sink_csv_data.y ),
        sink_csv_data.heat_diss_kw,
        'kW',
        srid,
        ntw_id,
        ntw_graph_id )

Add pipe junctions to the network. Use function `write_junction_to_db` from package `dblayer.helpers.utn.thermal_network`.

In [11]:
for junction_name, junction_csv_data in junctions_csv_data.iterrows():

    nodes_data[junction_name] = write_junction_to_db(
        db_access,
        junction_name,
        Point2D( junction_csv_data.x, junction_csv_data.y ),
        srid,
        ntw_id,
        ntw_graph_id
        )

Add all pipes to the network and connect them. Use function `write_round_pipe_to_db` from package `dblayer.helpers.utn.thermal_network`.

In [12]:
for pipe_name, pipe_csv_data in pipes_csv_data.iterrows():

    write_round_pipe_to_db(
        db_access,
        pipe_name,
        nodes_data[pipe_csv_data.from_node],
        nodes_data[pipe_csv_data.to_node],
        srid,
        ntw_id,
        ntw_graph_id,
        'distribution-pipe'
    )

Commit changes to database.

In [13]:
db_access.commit_citydb_session()

Done.